In [1]:
!pip install openai pdfplumber neo4j langchain-text-splitters langchain-community langchain-openai numpy -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 581.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 887.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 887.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318

In [2]:
#openai key
import os
import getpass
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:') #sk-proj-8YH86xCqTyCm3FeMmNSlT3BlbkFJn6hj7dJcuFafKYEadQTe

OpenAI API Key:··········


In [3]:
client = OpenAI()

In [4]:
#neo4j uri env
from neo4j import GraphDatabase
from langchain_community.graphs import Neo4jGraph

%env NEO4J_URI=neo4j+s://e27ae38f.databases.neo4j.io
%env NEO4J_USERNAME=neo4j
%env NEO4J_PASSWORD=MLps-fiUzCGYlPY8pUoNX0IFgWnlpyxBLFph1W8sf3Y

uri = "neo4j+s://e27ae38f.databases.neo4j.io"
user = "neo4j"
password = "MLps-fiUzCGYlPY8pUoNX0IFgWnlpyxBLFph1W8sf3Y"

graph = Neo4jGraph()
driver = GraphDatabase.driver(uri, auth=(user, password))

env: NEO4J_URI=neo4j+s://e27ae38f.databases.neo4j.io
env: NEO4J_USERNAME=neo4j
env: NEO4J_PASSWORD=MLps-fiUzCGYlPY8pUoNX0IFgWnlpyxBLFph1W8sf3Y


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/KGC'

/content/drive/MyDrive/KGC


## **Data Preprocessing** <br>
- keyword generation

In [ ]:
#pdf document import
import pdfplumber

In [ ]:
#extract text from pdf by pages
def extract_text_from_pdf(pdf_path):
    text_by_page = []
    page_count = 0
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            page_count += 1
            if text:
                text_by_page.append(text)
        # print("total page count: ",page_count)
    return text_by_page

pdf_path = 'apple_2023_1-20.pdf'
extraction = extract_text_from_pdf(pdf_path)
# print("extraction sample: ",extraction[4])

In [ ]:
#keyword extraction by pages using openai embedding
def extract_keywords(text):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "This document is mainly about company’s annual business report. Extract valid main keywords that could represent the document. Do not use any of markdowns or bullet points."},
                {"role": "user", "content": text}
            ],
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

keywords_list = [extract_keywords(page) for page in extraction]

for i, keywords in enumerate(keywords_list, 1):
    print(f"Page {i}:\n {keywords}")

Page 1:
 Apple Inc., annual business report, Securities and Exchange Commission, SEC, Form 10-K, fiscal year ended September 30, 2023, common stock, Nasdaq, trading symbols, notes, financial filing.
Page 2:
 annual business report, large accelerated filer, accelerated filer, non-accelerated filer, smaller reporting company, emerging growth company, Exchange Act, extended transition period, comply financial accounting standards, Section 13(a), Exchange Act, internal control, financial reporting, Section 404(b), Sarbanes-Oxley Act, registered public accounting firm, audit report, Section 12(b), Act, financial statements, correction of error, previously issued financial statements, error corrections, restatements, recovery analysis, incentive-based compensation, executive officers, registrant, relevant recovery period, §240.10D-1(b), shell company, Rule 12b-2, Act, market value, voting stock, non-voting stock, non-affiliates, fiscal quarter, business day, completed fiscal quarter, shares 

In [ ]:
#consolidate all the keywords into main keywords
def extract_overall_keywords(keywords_list):
    try:
        combined_keywords = ' '.join(keywords_list)

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Given keywords list is extracted from each pages of annual business report. Please boil down all these keywords into top important keywords that could encompass other words and closely related keywords. You can change the expression if you need. Don’t include keywords Apple Inc., annual business report, fiscal year 2023, form 10-k. And do not use any of markdowns or bullet points."},
                {"role": "user", "content": combined_keywords}
            ],
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
final_keywords = extract_overall_keywords(keywords_list).split(',')
print(final_keywords)

SEC compliance, stock trade, financial statements, market conditions, risk management, corporate governance, legal proceedings, global operations, intellectual property, supply chain, competition, product innovation, financial performance, cybersecurity, regulatory challenges, human capital, environmental impact, data privacy, customer experience, investment strategies, litigation risks, economic factors.


- Text splitting

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
#text split used for source text data injection to Neo4j
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=200
    )

splitted_texts = text_splitter.create_documents(extraction)

In [ ]:
# print(len(splitted_texts))
# print(splitted_texts[10])

- Neo4j injection

In [8]:
# from neo4j import GraphDatabase
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain_community.graphs import Neo4jGraph
from neo4j.exceptions import ClientError
from langchain_core.prompts import ChatPromptTemplate
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
llm = ChatOpenAI(temperature=0)
embeddings = OpenAIEmbeddings()
embedding_dimension = 1536

# graph = Neo4jGraph()

# uri = "neo4j+s://e27ae38f.databases.neo4j.io"
# user = "neo4j"
# password = "MLps-fiUzCGYlPY8pUoNX0IFgWnlpyxBLFph1W8sf3Y"

# driver = GraphDatabase.driver(uri, auth=(user, password))

In [ ]:
#insert splitted text data into nodes of Neo4j
for i, source in enumerate(splitted_texts):
    params = {
        "source_text": source.page_content,
        "source_id": i,
        "source_embedding": embeddings.embed_query(source.page_content),
            }
    #create nodes with vector embedding
    graph.query(
        """
    MERGE (p:Source {id: $source_id})
    SET p.text = $source_text
    WITH p
    CALL db.create.setVectorProperty(p, 'embedding', $source_embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )

    #create vector index for source data
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('source', "
            "'Source', 'embedding', $dimension, 'cosine')",
            {"dimension": embedding_dimension},
        )
    except ClientError:  #raise error if already exists
        pass

In [ ]:
#insert keywords into nodes of Neo4j
for i, keyword in enumerate(final_keywords):
    params = {
        "keyword_id": i,
        "keyword_text": keyword,
        "keyword_embedding": embeddings.embed_query(keyword),
            }
    #create nodes with vector embedding
    graph.query(
        """
    MERGE (k:Keyword{id: $keyword_id})
    SET k.text = $keyword_text
    WITH k
    CALL db.create.setVectorProperty(k, 'embedding', $keyword_embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )

    #create vector index for source data
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('keyword', "
            "'Keyword', 'embedding', $dimension, 'cosine')",
            {"dimension": embedding_dimension},
        )
    except ClientError:  #raise error if already exists
        pass

In [ ]:
# question generation and node injection
class Questions(BaseModel):
#     """Generating hypothetical questions about text."""

    questions: List[str] = Field(
        ...,
        description=(
            "Generated hypothetical questions based on the information from the text"
        ),
    )


questions_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating hypothetical questions based on the information found in the text. Make sure to provide full context in the generated questions."
            ),
        ),
        (
            "human",
            (
                "Use the given format to generate hypothetical questions from the following input: {input}"
            ),
        ),
    ]
)

question_chain = questions_prompt | llm.with_structured_output(Questions)
for i, source in enumerate(splitted_texts):
    questions = question_chain.invoke(source.page_content).questions
    params = {
        "source_id": i,
        "source_text": source.page_content,
        "questions": [
            {"text": q, "id": f"{i}-{iq}", "embedding": embeddings.embed_query(q)}
            for iq, q in enumerate(questions)
            if q
        ],
    }
    graph.query(
        """
    MERGE (p:Source {id: $source_id,text: $source_text})
    WITH p
    UNWIND $questions AS question
    CREATE (q:Question {id: question.id})
    SET q.text = question.text
    MERGE (q)<-[:HAS_QUESTION]-(p)
    WITH q, question
    CALL db.create.setVectorProperty(q, 'embedding', question.embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('hypothetical_questions', "
            "'Question', 'embedding', $dimension, 'cosine')",
            {"dimension": embedding_dimension},
        )
    except ClientError:  # already exists
        pass

In [ ]:
#summary generation and node injection
summary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are generating concise and accurate summaries based on the information found in the text."
            ),
        ),
        (
            "human",
            ("Generate a summary of the following input: {input}\n" "Summary:"),
        ),
    ]
)

summary_chain = summary_prompt | llm

for i, source in enumerate(splitted_texts):
    summary = summary_chain.invoke({"input": source.page_content}).content
    params = {
        "summary_id": i,
        "summary_text": summary,
        "embedding": embeddings.embed_query(summary),
    }
    graph.query(
        """
    MERGE (p:Source {id: $summary_id})
    MERGE (p)-[:HAS_SUMMARY]->(s:Summary)
    SET s.text = $summary_text
    WITH s
    CALL db.create.setVectorProperty(s, 'embedding', $embedding)
    YIELD node
    RETURN count(*)
    """,
        params,
    )
    # Create vector index
    try:
        graph.query(
            "CALL db.index.vector.createNodeIndex('summary', "
            "'Summary', 'embedding', $dimension, 'cosine')",
            {"dimension": embedding_dimension},
        )
    except ClientError:  #raise error if already exists
        pass

- keyword matching with questions

In [ ]:
import numpy as np

In [ ]:
def create_similarity_relationship(tx, question_id, keyword_id, similarity):
    query = """
    MATCH (q:Question {id: $question_id})
    MATCH (k:Keyword {id: $keyword_id})
    MERGE (q)-[r:IS_SIMILAR]->(k)
    SET r.similarity = $similarity
    """
    tx.run(query, question_id=question_id, keyword_id=keyword_id, similarity=similarity)

In [ ]:
#retrieve Question embeddings and Keyword embeddings
def fetch_embeddings(tx):
    query = """
    MATCH (q:Question)
    MATCH (k:Keyword)
    RETURN q.text AS questionText, q.id AS questionId, q.embedding AS questionEmbedding,
           collect({id: k.id, text: k.text, embedding: k.embedding}) AS keywords
    """
    return list(tx.run(query))

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

def find_most_similar_keyword(question_embedding, keywords):
    max_similarity = -np.inf
    most_similar_keyword = None
    keyword_id = None
    for keyword in keywords:
        if keyword['embedding'] is None:
            continue
        similarity = cosine_similarity(np.array(question_embedding), np.array(keyword['embedding']))
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_keyword = keyword['text']
            keyword_id = keyword['id']
    return most_similar_keyword, keyword_id, max_similarity

def main():
    with driver.session() as session:
        questions_and_keywords = session.execute_read(fetch_embeddings)


In [ ]:
with driver.session() as session:
    questions_and_keywords = session.execute_read(fetch_embeddings)
    for matching in questions_and_keywords:
      question_text = matching['questionText']
      question_id = matching['questionId']
      question_embedding = matching['questionEmbedding']
      keywords = matching['keywords']
      if question_embedding and keywords:
        most_similar_keyword, keyword_id, similarity = find_most_similar_keyword(question_embedding, keywords)
        if most_similar_keyword:
          session.execute_write(create_similarity_relationship,question_id,keyword_id, similarity)

- Retriever

In [64]:
from langchain_community.vectorstores import Neo4jVector
from langchain_core.runnables import ConfigurableField, RunnableParallel
from langchain_core.output_parsers import StrOutputParser


In [9]:
typical_rag = Neo4jVector.from_existing_index(
    OpenAIEmbeddings(), index_name="source"
)

In [11]:
#search the relevant keyword
keyword_vectorstore = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    index_name="keyword",
    node_label="Keyword",
    text_node_properties=["text","id"],
    embedding_node_property="embedding"
)
query="when is the end of fiscal year of apple?"
keyword_result = keyword_vectorstore.similarity_search(query, k=1)
print(keyword_result[0])

page_content='
text:  financial performance
id: 12'


In [47]:
#search the relevant question from retrieved keyword
retrieved_keyword = str(keyword_result[0])
# print(retrieved_keyword)

start = retrieved_keyword.find('text:') + len('text:')
end = retrieved_keyword.find('\nid:', start)
keyword_text = retrieved_keyword[start:end].strip()
print(keyword_text)

start = retrieved_keyword.find('id:') + len('id:')
end = retrieved_keyword.find("'", start)
keyword_id = int(retrieved_keyword[start:end].strip())
print(keyword_id)

financial performance
12


In [55]:
#retrieve relevant questions from retrieved keyword
cypher_query = """
      MATCH (k:Keyword{id:$keyword_id_path})-[]-(q:Question)
      RETURN k.id AS keywordId, q.id AS questionId, q.text AS questionText
      """

with driver.session() as session:
  cypher_result = session.run(cypher_query,keyword_id_path=int(keyword_id))
  # print(cypher_result.data())
  for relevant in cypher_result:
    print(relevant)

<Record keywordId=12 questionId='5-4' questionText='How does the Company manage its business segments geographically?'>
<Record keywordId=12 questionId='6-1' questionText="How does competition impact the Company's gross margins?">
<Record keywordId=12 questionId='9-0' questionText="How do seasonal holiday demands impact the Company's net sales in the first quarter compared to other quarters?">
<Record keywordId=12 questionId='9-1' questionText="How do new product and service introductions affect the Company's net sales, cost of sales, and operating expenses?">
<Record keywordId=12 questionId='9-3' questionText="How does anticipation of a product introduction affect the Company's net sales?">
<Record keywordId=12 questionId='9-7' questionText='How does the Company ensure that compensation is competitive and equitable for its employees?'>
<Record keywordId=12 questionId='11-0' questionText="How do global and regional economic conditions impact the Company's operations and performance?">


In [66]:
#retrieve from whole question nodes(keyword not specified)
cypher_query = """
      MATCH (k:Keyword{id:12})-[]-(q:Question)
      RETURN k.id AS keywordId, q.id AS questionId, q.text AS questionText,score
      """
question_vectorstore = Neo4jVector.from_existing_graph(
  OpenAIEmbeddings(),
  index_name="hypothetical_questions",
  keyword_index_name="keyword",
  node_label="Question",
  text_node_properties=["text","id"],
  embedding_node_property="embedding"
)
query="when is the end of fiscal year of apple?"

question_result_whole= question_vectorstore.similarity_search(query,k=2)
print(question_result_whole)

[Document(page_content='\ntext: What is the fiscal year end date for Apple Inc.?\nid: 0-0'), Document(page_content="\ntext: What are the key sections included in Apple Inc.'s Form 10-K for the Fiscal Year Ended September 30, 2023?\nid: 2-0")]


In [68]:
#retrieve question nodes from specified keyword that retrieved from pre-stage(keyword specified)
cypher_query = """
      MATCH (k:Keyword{id:$keyword_id_path})-[]-(q:Question)
      RETURN k.id AS keywordId, q.id AS questionId, q.text AS questionText,score
      """
question_vectorstore = Neo4jVector.from_existing_index(
  OpenAIEmbeddings(),
  index_name="hypothetical_questions",
  retrieval_query=cypher_query
)
query="when is the end of fiscal year of apple?"

question_result_wkey= question_vectorstore.similarity_search(query,k=2,params={"keyword_id_path":keyword_id})
print(question_result_wkey)  #no outputs..

KeyError: 'text'

In [63]:
retriever = typical_rag.as_retriever().configurable_alternatives(
    ConfigurableField(id="strategy"),
    default_key="typical_rag",
    keyword_strategy=keyword_vectorstore.as_retriever(),
    hypothetical_questions=question_vectorstore.as_retriever(),
)

In [65]:
from operator import itemgetter

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    RunnableParallel(
        {
            "context": itemgetter("question") | retriever ,
            "question": itemgetter("question"),
        }
    )
    | prompt
    | model
    | StrOutputParser()
)

In [72]:
if __name__ == "__main__":
    original_query = "when is the end of fiscal year of apple?"
    print(
        chain.invoke(
            {"question": original_query},
            {"configurable": {"strategy": "keyword_strategy"}},
        )
    )

The end of fiscal year of Apple is not provided in the given context.


In [71]:
if __name__ == "__main__":
    original_query = "when is the end of fiscal year of apple?"
    print(
        chain.invoke(
            {"question": original_query},
            {"configurable": {"strategy": "hypothetical_questions"}},
        )
    )

The fiscal year end date for Apple Inc. is September 30th.
